# OMR SHEET Evauation


In [18]:
import cv2
import numpy as np

# Load image as greyscale 
image = cv2.imread('omrSheet2F.png',0)
# Load image orginal 
imageOrginal = cv2.imread('omrSheet2F.png')

#show image
cv2.imshow('Original (GrayScale)', image)
#wait till any key press
cv2.waitKey(0)
#close all image screen
cv2.destroyAllWindows()



In [ ]:
# Scale image if required
'''
image = cv2.resize(image, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
imageOrginal = cv2.resize(imageOrginal, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)

cv2.imshow('Scaling (Interpolation)', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

In [2]:
# GaussianBlur to remove image 
blur = cv2.GaussianBlur(image.copy(), (5,5), 0)
cv2.imshow('Gaussian Blurring', blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# Values below 225 goes to 0 (black), everything above goes to 255 (white)
ret,thresh = cv2.threshold(blur, 225, 255, cv2.THRESH_BINARY)
cv2.imshow('Threshold Binary', thresh)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [4]:
# Canny Edge Detection
edged = cv2.Canny(blur, 75, 200)
cv2.imshow('Canny', edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# This will find out the OMR as a whole
# Find contours and print how many were found
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print ("Number of contours = ", len(contours))

contours = sorted(contours, key=cv2.contourArea, reverse=True)


Number of contours =  1


In [6]:
img = imageOrginal.copy()
sheetContour = None
   
#finding Rectangular Contour(Rectangular Sheet OMR)
for cnt in contours:
    
    # Get approximate polygons
    approx = cv2.approxPolyDP(cnt, 0.02*cv2.arcLength(cnt,True),True)
    
    if len(approx) == 4:    #if 4 dimentions means it will be rectangle
        sheetContour = cnt
        # determine the most extreme points along the contour
        extTopLeft  = (approx[0][0][0], approx[0][0][1])
        extBotLeft  = (approx[1][0][0], approx[1][0][1])
        extBotRight = (approx[2][0][0], approx[2][0][1])
        extTopRight = (approx[3][0][0], approx[3][0][1])
        break

        


In [7]:
# draw the outline of the object, then draw each of the
# extreme points
cv2.drawContours(img, [sheetContour], -1, (0, 255, 255), 2)
cv2.circle(img, extTopLeft, 20, (120, 120, 255), -1)
cv2.circle(img, extBotLeft,20, (65, 255, 120), -1)
cv2.circle(img, extBotRight, 20, (255, 90, 180), -1)
cv2.circle(img, extTopRight, 20, (255, 255, 0), -1)

# show the output image
cv2.imshow("Rectangular Contour(Rectangular Sheet OMR)", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#extTopLeft, extTopRight, extBotLeft, extBotRight
#extTopLeft, extBotLeft, extBotRight, extTopRight


In [8]:
cv2.imshow('Original', image)
cv2.waitKey(0)

# Cordinates of the 4 corners of the original image
points_A = np.float32([extTopLeft, extTopRight, extBotLeft, extBotRight])

# Cordinates of the 4 corners of the desired output
points_B = np.float32([[0,0], [420,0], [0,594], [420,594]])
 
# compute the Perspective Transformation matrix   
matrix = cv2.getPerspectiveTransform(points_A, points_B)
warpedOrginal = cv2.warpPerspective(imageOrginal, matrix, (420,594)) 
warped = cv2.warpPerspective(image, matrix, (420,594))
 
cv2.imshow('warpPerspective', warpedOrginal)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# Use erosion if required 
'''# Let's define our kernel size
kernel = np.ones((5,5), np.uint8)

# Now we erode
erosion = cv2.erode(warped.copy(), kernel, iterations = 1)
cv2.imshow('Erosion', erosion)
cv2.waitKey(0)
'''
# if not using erosion then just copy above value
erosion = warped.copy()

#use second argument as per requirement or accoring to document B/w ratio 
_,warpedThresh = cv2.threshold(erosion.copy(), 250, 255,cv2.THRESH_BINARY_INV )
cv2.imshow('Threshold Binary', warpedThresh)
cv2.waitKey(0) 
cv2.destroyAllWindows()

# Canny Edge Detection
warpedEdged = cv2.Canny(warpedThresh.copy(), 75, 200)
cv2.imshow('Canny', warpedEdged)
cv2.waitKey(0)

cv2.destroyAllWindows()

In [10]:
# Find contours inside the sheet (Only Focus on circular once with perticular diameter)

contours, hierarchy = cv2.findContours(warpedEdged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print ("Total Number of contours = ", len(contours))


circularCnts = []
for c in contours:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    # in order to label the contour as a bubble, region
    # should be sufficiently wide, sufficiently tall, and
    # have an aspect ratio approximately equal to 1
    if w >= 25 and h >= 25 and ar >= 0.8 and ar <= 1.2:
        circularCnts.append(c)

print ("Circular contours = ", len(circularCnts))


target = warped.copy();
for c in circularCnts:
    cv2.drawContours(target, [c], -1, (0,255,255), 3)        

cv2.imshow('circular Contours', target)
cv2.waitKey()        
cv2.destroyAllWindows()

Total Number of contours =  72
Circular contours =  40


In [11]:
def x_cord_contour(contours):
    #Returns the X cordinate for the contour centroid
    M = cv2.moments(contours)
    return (int(M['m10']/M['m00']))
    
def y_cord_contour(contours):
    #Returns the Y cordinate for the contour centroid
    M = cv2.moments(contours)
    return (int(M['m01']/M['m00']))
    


# To sort question wise, use y as key first and then 
# sort n(no of options) bubble at a time using x as key

contours_top_to_bottom = sorted(circularCnts, key = y_cord_contour, reverse = False)


In [12]:
Quiz = warpedOrginal.copy()
ANSWER_KEY = {0: 1, 1: 2, 2: 0, 3: 3, 4: 1, 5: 1, 6: 2, 7: 0, 8: 3, 9: 1}

In [14]:
correct = 0
# each question has 4 possible answers, to loop over the
# question in batches of 4
for (q, i) in enumerate(np.arange(0, len(contours_top_to_bottom), 4)):
    # sort the contours for the current question from
    # left to right, then initialize the index of the bubbled answer
    
    cnts = sorted(contours_top_to_bottom[i:i + 4], key = x_cord_contour, reverse = False)
    bubbled = None
    
    # loop over the sorted contours
    for (j, c) in enumerate(cnts):
        # construct a mask that reveals only the current
        # "bubble" for the question
        mask = np.zeros(warpedThresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        # apply the mask to the thresholded image, then
        # count the number of non-zero pixels in the bubble area
        mask = cv2.bitwise_and(warpedThresh, warpedThresh, mask=mask)
        
        #cv2.drawContours(mask, [c], -1, 255, -1)
        #cv2.imshow('Outputly', warpedThresh)
        #cv2.imshow('Output', mask)
        cv2.waitKey()        
        cv2.destroyAllWindows()
        
        
        total = cv2.countNonZero(mask)
        #print(total)
        # if the current total has a larger number of total
        # non-zero pixels, then we are examining the currently bubbled-in answer
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)
        # initialize the contour color and the index of the

    color = (0, 0, 255) 
    k = ANSWER_KEY[q]   #ans of this ques
    #print(q)
    #print(k)
    # check to see if the bubbled answer is correct
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1
    # draw the outline of the correct answer on the test
    cv2.drawContours(Quiz, [cnts[k]], -1, color, 3)

In [17]:
# grab the test taker
score = (correct / 10.0) * 100
print("Percentage: {:.2f}%".format(score))
cv2.putText(Quiz, "Percentage: {:.2f}%".format(score), (0, Quiz.shape[1]+160),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", Quiz)
cv2.waitKey(0)
cv2.destroyAllWindows()

Percentage: 30.00%
